Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from random import choices, randint, choice, sample, random
from copy import copy, deepcopy

import lab9_lib
import numpy as np
import math


# Local Search Algorithm

### Mutation Techniques

In [2]:
def mutation1(ind):     # 1 loci(gene) mutated
    offspring = copy(ind)
    pos = randint(0, NUM_LOCI-1)
    offspring[pos] = 1 - offspring[pos]
    assert len(offspring) == NUM_LOCI
    return offspring

def mutation2(ind):     # Reset randomly a random number of loci
    offspring = copy(ind)
    poss = (randint(0, NUM_LOCI-1), randint(0, NUM_LOCI-1))
    offspring = ind[:min(poss)] + [choice([0, 1]) for _ in range(max(poss)-min(poss))] + ind[max(poss):]
    assert len(offspring) == NUM_LOCI
    return offspring

### Recombination Techniques

In [3]:
def one_cut_crossover(ind1, ind2):
    cut_point = randint(0, NUM_LOCI-1)
    offspring = ind1[:cut_point] + ind2[cut_point:]
    
    assert len(offspring) == NUM_LOCI
    return offspring

def two_cut_crossover(ind1, ind2):
    cut_points = (randint(0, NUM_LOCI-1), randint(0, NUM_LOCI-1))
    offspring = ind1[:min(cut_points)] + ind2[min(cut_points) : max(cut_points)] + ind1[max(cut_points):]
    
    assert len(offspring) == NUM_LOCI
    return offspring

def uniform_crossover(ind1, ind2):
    offspring = [ind1[i] if i % 2 else ind2[i] for i in range(NUM_LOCI)]

    assert len(offspring) == NUM_LOCI
    return offspring

def uniform_crossover_double(ind1, ind2):
    o1, o2 = ([ind1[i] if i % 2 else ind2[i]for i in range(NUM_LOCI)], [ind2[i] if i % 2 else ind1[i] for i in range(NUM_LOCI)])

    assert len(o1) == NUM_LOCI and len(o2) == NUM_LOCI
    return o1, o2

### Parent Selection

In [4]:
def roulette_wheel(population):     # Roulette wheel with same probability
    return choice(population)[0]

def variable_tournament(population):
    
    n_partecipants = randint(2, int(len(population)/2))
    pool = choices(population, k=n_partecipants)
    parent = max(pool, key=lambda ind: ind[1])

    return parent[0]

def static_tournament(population):

    pool = [choice(population) for _ in range(TOURNAMENT_SIZE)]
    champ = max(pool, key=lambda ind: ind[1])
    
    return champ[0]

def best_parent_ever(population):
    champ = max(population, key=lambda ind: ind[1])
    return champ[0]

def genotype_screening(population):     # Select the 2 element with most differnt genotype 
    distance = []
    for i in range(len(population)):
        distance.append([sum(np.bitwise_xor(np.array(population[i][0]), np.array(population[j][0]))) for j in range(len(population))])
    
    distance = np.array(distance)
    max_position = np.unravel_index(np.argmax(distance, axis=None), distance.shape)
    
    return population[max_position[0]][0], population[max_position[1]][0]

### Survival Selection

In [5]:
def survival_selection(population):
    population.sort(key=lambda ind: ind[1], reverse=True)       # ORDERING FROM BEST TO WORSE
    return population[:POPULATION_SIZE]       # SURVIVAL SELECTION
    
def remove_twin(population):        # Remove TWIN from the population because I belive that they will have the same fitness
    twins = set()
    for i in range(len(population)):
        for j in range(i+1, len(population)):
            if population[i]==population[j]:
                twins.add(j)
    
    new_p = [ind for i, ind in enumerate(population) if i not in twins]
    return new_p

def survive_only_the_best(population):
    population.sort(key=lambda ind: ind[1], reverse=True)       # ORDERING FROM BEST TO WORSE
    return population[:1]       # SURVIVAL SELECTION

### Island Definition

In [6]:
# MUTATION ISLAND #
mutation_strategy = [mutation1, mutation2]

def island_0(population):
    for _ in range(NUM_GENERATIONS):
        offspring = list()
        for _ in range(NEW_OFFSPRING):
            # MUTATION
            p = roulette_wheel(population)
            o = choice(mutation_strategy)(p)
            offspring.append((o, fitness(o)))

        population.extend(offspring)
        population = remove_twin(population)
        population = survival_selection(population)
    return population

In [7]:
# RECOMBINATION ISLAND #
recombination_strategy = [one_cut_crossover, two_cut_crossover, uniform_crossover]
w = [0.5, 0.3, 0.1]

def island_1(population):
    for _ in range(NUM_GENERATIONS):
        offspring = list()
        for _ in range(NEW_OFFSPRING):
            #CROSS-OVER
            p1 = roulette_wheel(population)
            p2 = roulette_wheel(population)
            o =  choices(recombination_strategy, weights=w, k=1)[0](p1, p2)
            offspring.append((o, fitness(o)))

        population.extend(offspring)
        population = survival_selection(population)

    return population

In [8]:
# CHAMPS ISLAND #

def island_best_fitness(population):   # Island filled every round with the individual having the best fitness
    offspring = list()
    
    for _ in range(NEW_OFFSPRING):
        #CROSS-OVER
        p1 = roulette_wheel(population)
        p2 = roulette_wheel(population)
        o =  uniform_crossover(p1, p2)
        offspring.append((o, fitness(o)))
    
    offspring = survival_selection(offspring)

    return offspring

In [9]:
# RANDOM ISLAND # 

def island_random(population):   # Island filled every round with one random individual from the two original island
    offspring = list()
    for _ in range(NEW_OFFSPRING):
        if random() < MUTATION_PROBABILITY:
            # MUTATION 
            p = roulette_wheel(population)
            o = mutation1(p)
        else:
            #CROSS-OVER
            p1 = variable_tournament(population)
            p2 = variable_tournament(population)
            o =  choices(recombination_strategy, weights=w, k=1)[0](p1, p2)
        offspring.append(o)

    population.extend(offspring)
    population = remove_twin(population)
    population = survival_selection(population)
    return population

In [10]:
# SOLITUDE ISLAND #
mutation_strategy = (mutation1, mutation2)
recombination_strategy = (one_cut_crossover, two_cut_crossover, uniform_crossover)
w = [0.5, 0.3, 0.1]

def island_alone(population):   # Island with 1 element, the best one that will: mutate if it's the only one; recombine if they are 2. At the end only 1 survive
    
    match len(population):
        case 1:
            # MUTATION
            o = choice(mutation_strategy)(population[0][0])
            population.append((o, fitness(o)))

        case 2:
            #CROSS-OVER
            p1 = population[0][0]
            p2 = population[1][0]
            o =  choices(recombination_strategy, weights=w, k=1)[0](p1, p2)
            population.append((o, fitness(o)))

        case _:
            print("Empty")

    population = survive_only_the_best(population)
    return population

## Problem start from HERE ->

In [11]:
# GLOBAL PARAMETER #
ERA = 5
NUM_GENERATIONS = 50
POPULATION_SIZE = 10

NEW_OFFSPRING = 30
TOURNAMENT_SIZE = 5
NUM_ISLANDS = 3

NUM_LOCI = 1000
MUTATION_PROBABILITY = .15

# . . . # Problem Definition + first individual # . . . #

fitness = lab9_lib.make_problem(1)
starting_population = [choices([0, 1], k=NUM_LOCI) for _ in range(2*POPULATION_SIZE)]

In [12]:
starting_population = [(ind, fitness(ind)) for ind in starting_population]
galapagos_population = [choices(starting_population, k=POPULATION_SIZE), choices(starting_population, k=POPULATION_SIZE), [], [choice(starting_population)]]       # Island 0, Island 1, Island 2, Island 3, Island 4 
recombination_strategy = [one_cut_crossover, two_cut_crossover, uniform_crossover]
parent_selection_strategy = (roulette_wheel, static_tournament, variable_tournament)
survival_selection_strategy = (remove_twin, survival_selection)


while galapagos_population[3][0][1]!=1:     # condizione da sistemareall(all(ind[1] != 1 for ind in population) for population in galapagos_population
    for era in range(ERA):
        # Island 0 #
        galapagos_population[0] = island_0(galapagos_population[0])
        galapagos_population[2].append(galapagos_population[0][0])

        # Island 1 #
        galapagos_population[1] = island_1(galapagos_population[1])
        galapagos_population[2].append(galapagos_population[1][0])

        # Island 3 #
        galapagos_population[3] = island_alone(galapagos_population[3])

        print(era,"\n", galapagos_population[0][0], "\n", galapagos_population[1][0])

    # Island 2 #
    galapagos_population[2] = island_best_fitness(galapagos_population[2])
    galapagos_population[3].append(galapagos_population[2][0])

    # Island 3 #
    galapagos_population[3] = island_alone(galapagos_population[3])
    
    print(galapagos_population[3][0][1], galapagos_population[3][0][0])
print(fitness.calls,"\n", galapagos_population[3][0][1])

0 
 ([1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1